In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from geopy.distance import geodesic
import os

In [59]:
# Defina o diretório onde os arquivos estão localizados

# tipo_dado = 'Diario'
# tipo_dado = 'Mensal'
tipo_dado = 'Anual'

diretorio = './base/GO/' + tipo_dado

# Carrega o DataFrame das distâncias uma vez, pois ele será utilizado várias vezes
df_distancia = pd.read_csv('./results/dist_estacoes.csv')

# Lista de arquivos para processar
arquivos = ['chuva.csv', 'ponto_orvalho.csv', 'pressao.csv', 'radiacao.csv', 'temperatura.csv', 'umidade.csv', 'vento_direcao.csv', 'vento_velocidade.csv']


In [60]:
for arquivo in arquivos:
    caminho_completo = os.path.join(diretorio, arquivo)
    
    # Extrair o nome da variável do nome do arquivo
    nome_variavel = arquivo.split('.')[0]  # Remove a extensão .csv
    

    print(caminho_completo)
    # Carregar o DataFrame a partir do arquivo
    df = pd.read_csv(caminho_completo)

    
    # Converter 'hr_medicao' para datetime e definir como índice do DataFrame, se aplicável
    if 'hr_medicao' in df.columns:
        df['hr_medicao'] = pd.to_datetime(df['hr_medicao'])
        df_pivoted = df.pivot_table(index='hr_medicao', columns='cdg_stacao', values=nome_variavel, aggfunc='mean')
        df_pivoted_not_null = df_pivoted.dropna()
        
    
    # Aqui você precisaria calcular df_correlacao para cada tipo de dado
    df_correlacao = df_pivoted_not_null.corr()
    # Suponha que df_correlacao já está calculado para este exemplo
    # Código para construir 'anotacoes' e criar o heatmap, assegurando que 'anotacoes' esteja definido

    # Cria um DataFrame para armazenar os textos de anotação que incluirão correlação e distância
    anotacoes = pd.DataFrame(index=df_correlacao.index, columns=df_correlacao.columns, dtype='object')

    for origem in df_correlacao.index:
        for destino in df_correlacao.columns:
            correlacao = df_correlacao.loc[origem, destino]
            # Buscar a distância correspondente
            distancia = df_distancia[(df_distancia['cdg_estacao_origem'] == origem) & (df_distancia['cdg_estacao_destino'] == destino)]['distancia_km'].values
            
            if distancia.size > 0:  # Se encontrou a distância
                texto_anotacao = f"{correlacao:.2f}\n{distancia[0]:.2f}km"
            else:
                texto_anotacao = f"{correlacao:.2f}\nN/A"
            anotacoes.loc[origem, destino] = texto_anotacao
    
    # Note que o título e o arquivo de saída devem refletir o tipo de dado sendo processado
    plt.figure(figsize=(10, 8))
    sns.heatmap(df_correlacao, annot=anotacoes.values, fmt="", cmap='coolwarm', vmin=-1, vmax=1)
    plt.title(f'Correlação {nome_variavel.capitalize()} entre estações')
    plt.savefig(f'./results/img/estacoes_correlacao/{tipo_dado}/Correção_{nome_variavel}.png')
    plt.close()  # Fecha a figura atual para que a próxima possa ser criada sem interferência

./base/GO/Anual/chuva.csv
./base/GO/Anual/ponto_orvalho.csv
./base/GO/Anual/pressao.csv
./base/GO/Anual/radiacao.csv
./base/GO/Anual/temperatura.csv
./base/GO/Anual/umidade.csv
./base/GO/Anual/vento_direcao.csv
./base/GO/Anual/vento_velocidade.csv
